# Process Mining

In questo file si trasforma la sequenza di transazioni generata con i task precedenti, in particolare nel task 5, in un file .xes o .sv in modo da poteli visualizzare come processi in Apromore.

In [ ]:
import nbimporter
from t1_ETLBasics import calories_to_df,sleep_to_df,exercise_to_df
from t3_TSToEvents import calories_to_events, sleep_to_events,exercise_to_events
from t4_FeatureRedaction import calories_kmeans, sleep_kmeans, exercise_kmeans
from t5_MakeTransactions import get_cal_transaction, get_sleep_transaction, get_exercise_transaction, get_master_transaction
from t6_SequenceMining import get_trans,get_list_m_transaction
PATH = '../../pmdata/'

In [ ]:
cal_transaction = get_cal_transaction(calories_kmeans(calories_to_events(calories_to_df(PATH,[1])),4))
sleep_transaction = get_sleep_transaction(sleep_kmeans(sleep_to_events(sleep_to_df(PATH,[1])),4))
ex_transaction = get_exercise_transaction(exercise_kmeans(exercise_to_events(exercise_to_df(PATH,[1])),4))

In [ ]:
def sorted_sequence(df):
    """Funzione che ordina per timestamp le sequenze"""
    for i in range(len(df)):
        df[i].sort(key = lambda x: x[0] )

In [ ]:
sorted_sequence(cal_transaction)
sorted_sequence(sleep_transaction)
sorted_sequence(ex_transaction)

In [ ]:
m_trans = get_master_transaction(cal_transaction,ex_transaction,sleep_transaction)

In [ ]:
main_transaction = get_list_m_transaction(m_trans)

In [ ]:
main_transaction

In [ ]:
flat_transactions = [ item for elem in main_transaction for item in elem]

In [ ]:

flat_transactions

In [ ]:
from xml.etree.ElementTree import Element, SubElement, Comment, tostring, ElementTree
from xml.dom import minidom
from datetime import *
def time_string(date):
    return date.strftime("%Y-%m-%d, %H:%M:%S")

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [ ]:
root = Element("log", {'xes.version':"2.0", 'xes.features':"nested-attributes", 'openxes.version':"2.27"})
root

In [ ]:
def current_tree():
    print(prettify(root))
current_tree() 


In [ ]:
SubElement(root, "extension", name="Lifecycle", prefix="lifecycle", uri="http://www.xes-standard.org/lifecycle.xesext" )

In [ ]:
SubElement(root, "extension", name="Organizational", prefix="org", uri="http://www.xes-standard.org/org.xesext" )
SubElement(root, "extension", name="Time", prefix="time", uri="http://www.xes-standard.org/time.xesext" )
SubElement(root, "extension", name="Concept", prefix="concept", uri="http://www.xes-standard.org/concept.xesext" )
SubElement(root, "extension", name="Cost", prefix="cost", uri="http://www.xes-standard.org/cost.xesext" )

In [ ]:
current_tree() 

In [ ]:
traces = [ SubElement(root,"trace") for _ in flat_transactions ]

In [ ]:
traces

In [ ]:
for i, trace in enumerate(traces):
    SubElement(trace,"string", key="concept:name", value=f'trace_{i}' )
    for item in flat_transactions[i]:
        e = SubElement(trace,"event")
        SubElement(e, "date", key="time:timestamp", value= time_string(item[0]) )
        SubElement(e,"string", key="concept:name", value=item[1] )
        SubElement(e,"string", key="org:group", value="partecipant" )
        SubElement(e,"string", key="lifecycle:transition", value="complete" )

In [ ]:
with open('test.xes', 'w') as f:
    f.write(prettify(root))